In [10]:
from selenium import webdriver
import re, bs4, time,datetime
import pandas as pd

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(executable_path="C:/Users/Shefali/Downloads/Installs/chromedriver_win32/chromedriver.exe")
driver.implicitly_wait(10)

title = []
author = []
cover_type = []
actual_price = []
discounted_price = []
current_date = []
img_url = []
product_url = []

for i in range(1,801):
    driver.get("https://www.booksmela.com/literature-fiction-books?p="+str(i))
    time.sleep(2)
    for element in driver.find_elements_by_xpath('//*[@id="ve_layered_container"]/div/ul/li/div/h2'):
        title.append(element.text)
    for element in driver.find_elements_by_xpath('//*[@id="ve_layered_container"]/div/ul/li/div/span'):
        author.append(element.text)  
    for element in driver.find_elements_by_xpath('//*[@id="ve_layered_container"]/div/ul/li/div/div[1]/p[2]'):
        actual_price.append(element.text)
    try:
        elements = driver.find_elements_by_xpath('//*[@id="ve_layered_container"]/div/ul/li/div/div[1]/p[1]')
        for el in elements:
            discounted_price.append(el.text)
    except NoSuchElementException:
        discounted_price = actual_price


    html_source = driver.page_source
    data = html_source.encode('utf-8')
#p = re.compile('\d.*')
    page = bs4.BeautifulSoup(data, 'lxml')
    containers = page.findAll('li',{'class':'item last'})


    for cont in containers:
        im = cont.find('img')['src']
        img_url.append(im)
        pg = cont.find('a')['href']
        product_url.append(pg)
        if datetime.datetime.today().time() < datetime.time(12,0,0):
            current_date.append(str(datetime.datetime.today().date()) + (' am'))
        else:
            current_date.append(str(datetime.datetime.today().date()) + (' pm'))


    '''
    for p in product_url:
        driver.get(p)
        if driver.find_element_by_xpath('//*[@id="product_addtocart_form"]/div[3]/div[4]/div[3]/span[2]/ul').text!=None:
            cover_info.append(driver.find_element_by_xpath('//*[@id="product_addtocart_form"]/div[3]/div[4]/div[3]/span[2]/ul').text)

    for d in range(len(cover_info)):
        cover_type.append(re.search('Binding: (.*)', cover_info[d]).group(1))
        cover_type = [ct.replace("Softcover","paperback") for ct in cover_type]
        cover_type = [ct.replace("No","paperback") for ct in cover_type]
        cover_type = [ct.replace("Hardcover","hard cover") for ct in cover_type]
        '''
    
    
cover_type = ['paperback'] * len(title)    
genre = ['fiction'] * len(title)

for i in range(0,len(title)):
    author[i] = author[i].replace("By: ","")
    actual_price[i] = actual_price[i].replace("₹","")
    actual_price[i] = actual_price[i].replace(",","")
    actual_price[i] = float(actual_price[i])
    discounted_price[i] = discounted_price[i].replace("₹","")
    discounted_price[i] = discounted_price[i].replace(",","")
    discounted_price[i] = float(discounted_price[i])

df_fiction = pd.DataFrame({'title':title,
                 'author':author,
                 'cover_type':cover_type,
                 'genre':genre,
                 'actual_price':actual_price,
                 'discounted_price':discounted_price,
                 'current_date':current_date,
                 'img_url':img_url,
                 'product_url':product_url})

p = re.compile('\.\d+')
a = datetime.datetime.today()
a = p.sub('',str(a))
a = a.replace(':','-')
df_fiction.to_csv("booksmela_fiction_"+ a +".csv",index=False)
driver.close()

df_fiction.head()

,title,author,cover_type,genre,actual_price,discounted_price,current_date,img_url,product_url
0,Das Din / दस दिन,Durvasa (Dr. O.P. Kejariwal) / दुर्वासा (डॉ. ओ...,paperback,fiction,240.0,228.0,2019-02-05 pm,https://www.booksmela.com/media/catalog/produc...,https://www.booksmela.com/das-din-durvasa-dr-o...
1,Aarakshan : A Fiction आरक्षण : एक उपन्यास,Durvasa (Dr. O.P. Kejariwal) दुर्वासा (डॉ. ओ.प...,paperback,fiction,150.0,143.0,2019-02-05 pm,https://www.booksmela.com/media/catalog/produc...,https://www.booksmela.com/aarakshan-a-fiction-...
2,Hues Shades Pricks: An Anthology of Poems,"Borgohain, C.",paperback,fiction,110.0,105.0,2019-02-05 pm,"data:image/webp;base64,UklGRhoJAABXRUJQVlA4IA4...",https://www.booksmela.com/hues-shades-pricks-a...
3,Redeemer,"Borkakoti, S.K.",paperback,fiction,395.0,375.0,2019-02-05 pm,https://www.booksmela.com/media/catalog/produc...,https://www.booksmela.com/redeemer-borkakoti-s...
4,Indira Goswami & V.S. naipaul,"Sharma, P.",paperback,fiction,625.0,593.0,2019-02-05 pm,https://www.booksmela.com/media/catalog/produc...,https://www.booksmela.com/indira-goswami-v-s-n...
